# mechanical turk
### from https://blog.mturk.com/tutorial-mturk-using-python-in-jupyter-notebook-17ba0745a97f

In [2]:
import boto3
import xmltodict
import json

In [3]:
create_hits_in_production = False
environments = {
  "production": {
    "endpoint": "https://mturk-requester.us-east-1.amazonaws.com",
    "preview": "https://www.mturk.com/mturk/preview"
  },
  "sandbox": {
    "endpoint": 
          "https://mturk-requester-sandbox.us-east-1.amazonaws.com",
    "preview": "https://workersandbox.mturk.com/mturk/preview"
  },
}
mturk_environment = environments["production"] if create_hits_in_production else environments["sandbox"]

session = boto3.Session(profile_name='mturk')
client = session.client(
    service_name='mturk',
    region_name='us-east-1',
    endpoint_url=mturk_environment['endpoint'],
)


In [5]:
print(client.get_account_balance()['AvailableBalance'])

10000.00


In [6]:
tweets = ['in science class right now... urgh... stupid project..',
          'hmmm what to have for breaky?... Honey on toast ',
          'Doing home work  x',
          'Headed out of town for a few days. Will miss my girls']

In [7]:
html_layout = open('./SentimentQuestion.html', 'r').read()
QUESTION_XML = """<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">
        <HTMLContent><![CDATA[{}]]></HTMLContent>
        <FrameHeight>650</FrameHeight>
        </HTMLQuestion>"""
question_xml = QUESTION_XML.format(html_layout)

In [8]:
TaskAttributes = {
    'MaxAssignments': 5,           
    # How long the task will be available on MTurk (1 hour)     
    'LifetimeInSeconds': 60*60,
    # How long Workers have to complete each item (10 minutes)
    'AssignmentDurationInSeconds': 60*10,
    # The reward you will offer Workers for each response
    'Reward': '0.05',                     
    'Title': 'Provide sentiment for a Tweet',
    'Keywords': 'sentiment, tweet',
    'Description': 'Rate the sentiment of a tweet on a scale of 1 to 10.'
}

In [9]:
results = []
hit_type_id = ''

for tweet in tweets:
    response = client.create_hit(
        **TaskAttributes,
        Question=question_xml.replace('${content}',tweet)
    )
    hit_type_id = response['HIT']['HITTypeId']
    results.append({
        'tweet': tweet,
        'hit_id': response['HIT']['HITId']
    })
    
print("You can view the HITs here:")
print(mturk_environment['preview']+"?groupId={}".format(hit_type_id))

You can view the HITs here:
https://workersandbox.mturk.com/mturk/preview?groupId=3VIW2KSTJ2H5RY6DILQS1K4M09VLL0


In [10]:
for item in results:
    
    # Get the status of the HIT
    hit = client.get_hit(HITId=item['hit_id'])
    item['status'] = hit['HIT']['HITStatus']

    # Get a list of the Assignments that have been submitted
    assignmentsList = client.list_assignments_for_hit(
        HITId=item['hit_id'],
        AssignmentStatuses=['Submitted', 'Approved'],
        MaxResults=10
    )
    assignments = assignmentsList['Assignments']
    item['assignments_submitted_count'] = len(assignments)

    answers = []
    for assignment in assignments:
    
        # Retreive the attributes for each Assignment
        worker_id = assignment['WorkerId']
        assignment_id = assignment['AssignmentId']
        
        # Retrieve the value submitted by the Worker from the XML
        answer_dict = xmltodict.parse(assignment['Answer'])
        answer = answer_dict['QuestionFormAnswers']['Answer']['FreeText']
        answers.append(int(answer))
        
        # Approve the Assignment (if it hasn't been already)
        if assignment['AssignmentStatus'] == 'Submitted':
            client.approve_assignment(
                AssignmentId=assignment_id,
                OverrideRejection=False
            )
    
    # Add the answers that have been retrieved for this item
    item['answers'] = answers
    if len(answers) > 0:
        item['avg_answer'] = sum(answers)/len(answers)

print(json.dumps(results,indent=2))

[
  {
    "tweet": "in science class right now... urgh... stupid project..",
    "hit_id": "3ZQX1VYFTDBYSLH87VRZ3HTJMUA8OS",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": []
  },
  {
    "tweet": "hmmm what to have for breaky?... Honey on toast ",
    "hit_id": "3Z56AA6EK46NO4D810FORZ8NLZ56MP",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": []
  },
  {
    "tweet": "Doing home work  x",
    "hit_id": "3XAOZ9UYRZXJ2XA3NNAODC1BNPM1Q9",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": []
  },
  {
    "tweet": "Headed out of town for a few days. Will miss my girls",
    "hit_id": "3TD33TP5DL8SSFLE0O6RTOZF5JRBAV",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": []
  }
]
